# A Pythonic Object
## Object Representations

Every object-oriented language has at least one standard way of getting a string repre‐
sentation from any object. Python has two:
-  repr(): Return a string representing the object as the developer wants to see it.
-  str(): Return a string representing the object as the user wants to see it.

There are two additional special methods to support alternative representations of objects: __bytes__ and __format__. 

## Vector Class Redux

In [1]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y)) 
        
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self) 

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

myvec = Vector2d(7, 10)

In [6]:
for i in myvec: # using the __iter__
    print(i)

7.0
10.0


In [8]:
myvec.__repr__() # all the information we need to create this object

'Vector2d(7.0, 10.0)'

In [10]:
eval(myvec.__repr__()) == myvec

True

In [9]:
str(myvec) # a string representation of the object

'(7.0, 10.0)'

In [12]:
myvec.__bytes__()

b'd\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00$@'

In [14]:
Vector2d(1, 2) == Vector2d(1, 2) # using the __eq__

True

In [17]:
abs(myvec), math.hypot(7, 10) # using __abs__

(12.206555615733702, 12.206555615733702)

In [18]:
bool(myvec)

True

## classmethod Versus staticmethod
The staticmethod decorator changes a method so that it receives no special first argument. classmethod passes the class implicitly as the first argument.


In [21]:
class Demo:

    @classmethod
    def klassmeth(*args):
        return args

    @staticmethod
    def statmeth(*args):
        return args


In [22]:
Demo.klassmeth() # passes the class instance as 

(__main__.Demo,)

In [23]:
Demo.statmeth() # Demo.statmeth behaves just like a plain old function

()

The classmethod decorator is clearly useful, but I’ve never seen a compelling use case for staticmethod. If you want to define a
function that does not interact with the class, just define it in the module.

## Formatted Displays
The format() built-in function and the str.format() method delegate the actual formatting to each type by calling their .__format__(format_spec) method. 

In [25]:
brl = 1 / 2.43

In [26]:
brl

0.4115226337448559

In [27]:
format(brl, "0.4f")

'0.4115'

In [28]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

In [29]:
 format(42, 'b')

'101010'

In [30]:
format(2/3, '.1%')

'66.7%'

## A Hashable Vector2d
As defined, so far our Vector2d instances are unhashable, so we can’t put them in a set:

In [31]:
v1 = Vector2d(3, 4)

In [33]:
import traceback

try:
    hash(v1)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-33-b0a524b4faf1>", line 4, in <module>
    hash(v1)
TypeError: unhashable type: 'Vector2d'


To make a Vector2d hashable, we must implement __hash__ (__eq__ is also required, and we already have it). We also need to make vector instances immutable, as we’ve seen in 'What Is Hashable?' on page 65. As per the [Python glossary](https://docs.python.org/3/glossary.html): 'An object is hashable if it has a hash value which never changes during its lifetime (it needs a __hash__() method), and can be compared to other objects (it needs an __eq__() method). Hashable objects which compare equal must have the same hash value.' The atomic immutable types (str, bytes, numeric types) are all hashable.

In [3]:
class MoreVector2d(Vector2d):
    '''
    We need a __hash__ and an __eq__ method.
    __eq__ has already been defined in Vector2d.
    Here we add in __hash__.
    Really we should also make x and y private and unchangeable.
    The hash cannot change over the object's lifetime. 
    '''
    def __init__(self, x, y):
        super(MoreVector2d, self).__init__(x, y)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

myvec = MoreVector2d(3, 4)

In [4]:
myvec

MoreVector2d(3.0, 4.0)

In [5]:
hash(myvec)

7

## Private and protected attributes in Python
In Python there is no way to create  pricate variables like there is with the private modifier in Java. However, if you name an instance attribute with two leading underscores, Python stores the name in the instance __dict__ prefixed with a leading underscore and the class name. This language feature is called 'name mangling'. 

In [8]:
class MyClass(object):
    def __init__(self):
        self.a = 7
        self.__b = 8

obj = MyClass()
obj

In [9]:
obj.__dict__

{'a': 7, '_MyClass__b': 8}

This helps to prevent the accidental overwriting of a private attribute in a subclass.

In [19]:
class A(object):
    def __init__(self):
        self.a = 7
        self.__b = 8

class B(A):
    def __init__(self):
        super(B, self).__init__()
        self.a = 9
        self.__b = 10

b = B()
b

In [21]:
b.__dict__ # a gets overwritten, but b has values unique to each of classes A and B.

{'a': 9, '_A__b': 8, '_B__b': 10}

The name mangling functionality is not loved by all Pythonistas, and neither is the skewed look of names written as self.__x. Some prefer to avoid this syntax and use just one underscore prefix to 'protect' attributes by convention (e.g., self._x). Critics of the automatic double-underscore mangling suggest that concerns about accidental attribute clobbering should be addressed by naming conventions. This is the full quote from the prolific Ian Bicking, cited at the beginning of this chapter: <br>
<br>
Never, ever use two leading underscores. This is annoyingly private. If name clashes are a concern, use explicit name mangling instead (e.g., _MyThing_blahblah). This is essentially the same thing as double-underscore, only it’s transparent where double underscore obscures <br>
<br>
The single underscore prefix has no special meaning to the Python interpreter when used in attribute names, but it’s a very strong convention among Python programmers that you should not access such attributes from outside the class.

## Saving Space with the __slots__ Class Attribute
By default, Python stores instance attributes in a per-instance dict named __dict__. dictionaries have a significant memory overhead because of the underlying hash table used to provide fast access. By defining __slots__ in the class, you are telling the interpreter: 'These are all the instance attributes in this class.' Python then stores them in a tuple-like structure in
each instance, avoiding the memory overhead of the per-instance __dict__. 

In [1]:
class MyClass1(object):
    def __init__(self):
        self.x = 1
        self.y = 2
        self.z = 3


class MyClass2(object):
    __slots__ = ("x", "y", "z")
    def __init__(self):
        self.x = 1
        self.y = 2
        self.z = 3

obj1 = MyClass1()
obj2 = MyClass2()

In [5]:
from sys import getsizeof
getsizeof(obj1), getsizeof(obj2) # hmmm, is obj2 larger...

(56, 64)

In [6]:
%timeit [MyClass1() for i in range(1000000)]

355 ms ± 3.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit [MyClass2() for i in range(1000000)] # materially faster to create the object

264 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The creator of Python, Guido van Rossum, states that he actually created __slots__ for faster attribute access. It is trivial to demonstrate measurably significant faster access:

In [3]:
%timeit obj1.x

30 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [4]:
%timeit obj2.x

27 ns ± 0.178 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Small caveat, you should only declare a particular slot one time in an inheritance tree. For example:

In [9]:
class Base:
    __slots__ = 'foo', 'bar'

class Right(Base):
    __slots__ = 'baz', 

class Wrong(Base):
    __slots__ = 'foo', 'bar', 'baz'        # redundant foo and bar

Base(), Right(), Wrong()

(<__main__.Base at 0x21479a68748>,
 <__main__.Wrong at 0x21479a814f8>)

There is a long discussion on [SO](https://stackoverflow.com/questions/472000/usage-of-slots)... the book is taking the other approach though and saying that you must remember to redeclare __slots__ in each subclass, because the inherited attribute is ignored by the interpreter. See the docs.

## Overriding Class Attributes